For the purchase cost of equipment, the following code can be used to generate a dictionary

In [ ]:
import pandas as pd

purchase_df = pd.DataFrame({
    'Unit': [u.ID for u in my_sys.cost_units],                          # The system here is named my_sys
    'Purchase Cost': [u.purchase_cost for u in my_sys.cost_units]
})
purchase_df.to_excel('equipment_purchase_costs.xlsx', index=False)


To simply print these out, one can use:

In [ ]:
for u in my_sys.cost_units:
    print(f"{u.ID}: {u.purchase_cost}")